# 05_compute_ISC

In [ ]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

## define functions

In [ ]:
def load_sub_raw_data(data_folder='/nfs/e5/studyforrest/forrest_movie_meg/', subject_idx='01', run_idx='01'):
    """
    load raw meg data. 
    
    input value: subject_idx and run_idx should be str
    """
    
    if not isinstance(subject_idx, str):
        raise ValueError('subject_dix must be str')
        
    if not isinstance(run_idx, str):
        raise ValueError('run_idx must be str')
    
    subject_data_folder = data_folder + 'sub-' + subject_idx + '/ses-movie/meg'
    fname = 'sub-' + subject_idx + '_ses-movie_task-movie_run-' + run_idx + '_meg.ds'
    raw_data_path = pjoin(subject_data_folder, fname)
    raw_data = mne.io.read_raw_ctf(raw_data_path, preload='True')
    
    print('total channels number is {}'.format(len(raw_data.info['chs'])))
    print('sample frequency is {} Hz'.format(raw_data.info['sfreq']))

    return raw_data

def band_split(raw, band_name='gamma'):
    """
    band_name: "delta": 1-4Hz
               "theta": 4-8Hz
               "alpha": 8-13Hz
               "beta": 13-30Hz
               "gamma": 30-100Hz
    """
    
    if band_name == 'delta':
        raw_band = raw.copy().load_data().filter(l_freq=1, h_freq=4)
    elif band_name == 'theta':
        raw_band = raw.copy().load_data().filter(l_freq=4, h_freq=8)
    elif band_name == 'alpha':
        raw_band = raw.copy().load_data().filter(l_freq=8, h_freq=13)
    elif band_name == 'beta':
        raw_band = raw.copy().load_data().filter(l_freq=13, h_freq=30)
    elif band_name == 'gamma':
        raw_band = raw.copy().load_data().filter(l_freq=30, h_freq=100)
    
    return raw_band

def compute_band_intercorr(raw_sub1, raw_sub2, events_sub1, events_sub2, band_name):

    ch_name_picks1 = mne.pick_channels_regexp(raw_sub1.ch_names, regexp='M[LRZ]...-4503')
    type_picks1 = mne.pick_types(raw_sub1.info, meg=True)
    sub1_picks= np.intersect1d(ch_name_picks1, type_picks1)
    ch_name_picks2 = mne.pick_channels_regexp(raw_sub2.ch_names, regexp='M[LRZ]...-4503')
    type_picks2 = mne.pick_types(raw_sub2.info, meg=True)
    sub2_picks= np.intersect1d(ch_name_picks2, type_picks2)
    if len(sub1_picks) == len(sub2_picks):
        if (sub1_picks == sub2_picks).all():
            picks = sub1_picks
        else:
            picks = np.intersect1d(sub1_picks, sub2_picks)
        bad_idx = []
    else:
        picks = sub1_picks if len(sub1_picks) > len(sub2_picks) else sub2_picks
        bad_picks = np.union1d(np.setdiff1d(sub1_picks, sub2_picks), np.setdiff1d(sub2_picks, sub1_picks))
    
        bad_idx = []
        for chn in bad_picks:
            bad_idx.append(np.where(picks == chn)[0][0])
            
    raw_sub1_theta = band_split(raw_sub1, band_name=band_name)
    raw_sub2_theta = band_split(raw_sub2, band_name=band_name)
    sub1_band_data = raw_sub1_theta.get_data(picks=picks)
    sub2_band_data = raw_sub2_theta.get_data(picks=picks)
    
    sub1_envlope = np.abs(signal.hilbert(sub1_band_data))
    sub2_envlope = np.abs(signal.hilbert(sub2_band_data))
    
    if len(bad_idx) != 0:
        for idx in bad_idx:
            sub1_envlope[idx,:] = 0
            sub2_envlope[idx,:] = 0
    
    #downsampling
    sample_sub1 = events_sub1[0][1:-1:25,0]
    sample_sub2 = events_sub2[0][1:-1:25,0]
    sub1_band_dsamp = sub1_envlope.take(sample_sub1, axis=1)
    sub2_band_dsamp = sub2_envlope.take(sample_sub2, axis=1)
    
                
    #def hilbert_filter(x, sfreq, order=201):
    #    
    #    co = [2*np.sin(np.pi*n/2)**2/np.pi/n for n in range(1, order+1)]
    #    co1 = [2*np.sin(np.pi*n/2)**2/np.pi/n for n in range(-order, 0)]
    #    co = co1+[0]+ co
    #    # out = signal.filtfilt(b=co, a=1, x=x, padlen=int((order-1)/2))
    #    out = signal.convolve(x, co, mode='same', method='direct')
    #    envolope = np.sqrt(out**2 + x**2)
    #    
    #    return envolope
        
    ch_r = []
    ch_p = []
    for ch_idx in np.arange(sub1_band_dsamp.shape[0]):
        x = sub1_band_dsamp[ch_idx, :]
        y = sub2_band_dsamp[ch_idx, :]
        
        r, p = stats.pearsonr(x, y)
        ch_r.append(r)
        ch_p.append(p)
    
    del raw_sub1, raw_sub2
    
    return ch_r, ch_p

def extract_events(sub_idx, run_idx):
    bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/preproc_data'
    sub_path = BIDSPath(subject=sub_idx, run=int(run_idx), task='movie', session='movie', processing='preproc', root=bids_root)
    sub_raw = mne_bids.read_raw_bids(sub_path)
    events_sub = mne.events_from_annotations(sub_raw)
    
    return events_sub


## define variables

In [ ]:
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/preproc_data'
sub_list = np.arange(1,12)
run_list = np.arange(1,9)
save_pth = '/nfs/s2/userhome/daiyuxuan/workingdir/MEG-paper/output_data'
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']

## compute pre-post preproc ISC

In [ ]:
# pre pre-proc
pre_corr_band_split = {}
for band_name in band_names:
    
    pre_corr_band_split[band_name] = np.zeros((len(run_list), len(sub_list), len(sub_list), 272))
            
    for run in run_list:
        if run >= 7:
            sub_ls = np.arange(2,12)
        else:
            sub_ls = sub_list
            
        for sub1 in sub_ls:
            if sub1 < 10:
                sub1_idx = '0' + str(sub1)
            else:
                sub1_idx = str(sub1)
                
            run_idx = '0'+str(run)
            events_sub1 = extract_events(sub1_idx, run_idx)
            raw_sub1 = load_sub_raw_data(subject_idx=sub1_idx, run_idx=run_idx)

            for sub2 in sub_list:
                if sub2 < 10:
                        sub2_idx = '0' + str(sub2)
                else:
                        sub2_idx = str(sub2)
                
                events_sub2 = extract_events(sub2_idx, run_idx)
                raw_sub2 = load_sub_raw_data(subject_idx=sub2_idx, run_idx=run_idx)
                ch_r, ch_p = compute_band_intercorr(raw_sub1, raw_sub2, events_sub1, events_sub2, band_name)
                pre_corr_band_split[band_name][run-1][sub1-1][sub2-1] = pre_corr_band_split[band_name][run-1][sub2-1][sub1-1] = np.array(ch_r)
np.save(pjoin(save_pth, 'pre_corr_band_split'), pre_corr_band_split)

# post pre-proc
corr_band_split = {}
for band_name in band_names:
    
    corr_band_split[band_name] = np.zeros((len(run_list), len(sub_list), len(sub_list), 272))
    
    for run in run_list:
        if run >= 7:
            sub_ls = np.arange(2, 12)
        else:
            sub_ls = sub_list
        
        for sub1 in sub_ls:
            if sub1 < 10:
                sub1_idx = '0' + str(sub1)
            else:
                sub1_idx = str(sub1)
            
            sub1_path = BIDSPath(subject=sub1_idx, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
            raw_sub1 = mne_bids.read_raw_bids(sub1_path)
            events_sub1 = mne.events_from_annotations(raw_sub1)
            
            for sub2 in np.arange(sub1+1, 12):
                
                if sub2 < 10:
                    sub2_idx = '0' + str(sub2)
                else:
                    sub2_idx = str(sub2)
                
                sub2_path = BIDSPath(subject=sub2_idx, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
                raw_sub2 = mne_bids.read_raw_bids(sub2_path)
                events_sub2 = mne.events_from_annotations(raw_sub2)
                ch_r, ch_p = compute_band_intercorr(raw_sub1, raw_sub2, events_sub1, events_sub2, band_name)
                corr_band_split[band_name][run-1][sub1-1][sub2-1] = corr_band_split[band_name][run-1][sub2-1][sub1-1] = np.array(ch_r)
np.save(pjoin(save_pth, 'corr_band_split'), corr_band_split)
